# DVA263 project

# Libraries

Declaring all libraries in use

In [1]:
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
import os
import cnn as cnn
import preprocessing as pre
import visualization as vis
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns



# Load Dataset

In [2]:
def load_cnn():
    """Load the datasets for the CNN input"""
    CNN_train = "../dataset/CNN_Input/train/"
    CNN_test = "../dataset/CNN_Input/test/"
    CNN_train = pre.import_images(CNN_train, (150,150),batch_size=16, dataset=True)
    CNN_test = pre.import_images(CNN_test, (150,150), batch_size=16,  dataset=True)
    return CNN_train, CNN_test

# Prepare Data

In [3]:
def prep_cnn(cnn_train, cnn_test):
    """Add noise images to train and test set and normalize"""
    g_noisy_train_data = cnn_train.map(pre.add_gaussian_noise)
    snp_noisy_train_data = cnn_train.map(pre.add_salt_and_pepper_noise)
    combined_train_data = cnn_train.concatenate(g_noisy_train_data).concatenate(snp_noisy_train_data)

    percentage_noisy_images = 0.2
    total_test_images = len(CNN_test) * 16  # Total images in test data
    num_noisy_images = int(total_test_images * percentage_noisy_images)

    g_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_gaussian_noise)
    snp_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_salt_and_pepper_noise)
    remaining_test_data = cnn_test.skip(num_noisy_images)

    combined_test_data = g_noisy_test_data.concatenate(remaining_test_data)
    combined_test_data = combined_test_data.concatenate(snp_noisy_test_data)

    test_data_norm, train_data_norm = pre.normalize_dataset(combined_train_data, combined_test_data)
    return train_data_norm, test_data_norm


# Build and Compile Model

In [4]:
def build_cnn():
    model = cnn.build_cnn_model()
    return model

#  Train Model

In [5]:
def train_cnn(model,cnn_train, cnn_test):
    history = cnn.train_model(model, cnn_train, cnn_test)
    return history

# Putting everything together

Main section putting each part together and running the classification task

In [6]:
CNN_train, CNN_test = load_cnn()
CNN_train, CNN_test = prep_cnn(CNN_train, CNN_test)
CNN_model = build_cnn()
CNN_history = train_cnn(CNN_model, CNN_train, CNN_test)
CNN_model.predict(CNN_test) # here we will use the output from the autoencoder.


Found 83 files belonging to 4 classes.
Found 62 files belonging to 4 classes.
Epoch 1/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 151ms/step - accuracy: 0.2540 - loss: 1.3963 - val_accuracy: 0.3548 - val_loss: 1.3816 - learning_rate: 0.0010
Epoch 2/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.3692 - loss: 1.3802 - val_accuracy: 0.4597 - val_loss: 1.3768 - learning_rate: 0.0010
Epoch 3/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - accuracy: 0.3781 - loss: 1.3740 - val_accuracy: 0.3387 - val_loss: 1.3697 - learning_rate: 0.0010
Epoch 4/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 131ms/step - accuracy: 0.3386 - loss: 1.3606 - val_accuracy: 0.3548 - val_loss: 1.3495 - learning_rate: 0.0010
Epoch 5/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - accuracy: 0.3556 - loss: 1.3510 - val_accuracy: 0.3790 - val_loss: 1.3119 - learning_rate: 0.0010
Epoch 6/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.4079 - loss: 1.2825 - val_accuracy: 0.4758 - val_loss: 1.2216 - learning_rate: 0.0010
Epoch 7/70

array([[4.85610826e-06, 2.63648019e-07, 9.56072330e-01, 4.39225733e-02],
       [2.44411002e-12, 1.44989631e-10, 9.99945521e-01, 5.44608447e-05],
       [1.38029179e-07, 1.84997226e-08, 9.83583689e-01, 1.64162777e-02],
       [9.99958813e-01, 2.16068656e-05, 2.12348569e-10, 1.96711735e-05],
       [1.72418222e-08, 6.64652500e-10, 9.99999046e-01, 9.06875698e-07],
       [3.46184570e-05, 2.15672549e-06, 8.63600988e-03, 9.91327226e-01],
       [1.48184539e-04, 9.99839485e-01, 3.81842868e-09, 1.24283833e-05],
       [7.70107654e-05, 9.99765277e-01, 2.50317214e-07, 1.57408780e-04],
       [9.99998450e-01, 1.55385499e-06, 1.42676981e-09, 1.00995416e-08],
       [1.09340840e-06, 4.95249779e-08, 9.96014655e-01, 3.98417795e-03],
       [9.03423654e-07, 9.99999046e-01, 3.03760350e-09, 6.71848037e-12],
       [9.99812007e-01, 1.88015838e-04, 1.54452753e-11, 5.82446891e-09],
       [1.41216930e-07, 2.02068340e-09, 9.99989152e-01, 1.07253745e-05],
       [2.15314867e-05, 5.41578038e-06, 9.76149976e